# Crop

Wrap the `crop` step as a Common Workflow Language CommandLineTool and execute it with a CWL runner.

This notebook is linked to: https://eoap.github.io/mastering-app-package/cwl-cli/norm-diff/



## Setup

In [5]:
export WORKSPACE=/workspace/mastering-app-package
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
rm -fr ${RUNTIME}/*
cd ${RUNTIME}


## Run the crop

Inspect and use `cwltool` to run the CommandLineTool definition:

The CWL document below shows the `crop` step wrapped as a CWL CommandLineTool:

In [6]:

cat ${WORKSPACE}/cwl-cli/crop.cwl | yq e -

cwlVersion: v1.0
class: CommandLineTool
id: crop
requirements:
  InlineJavascriptRequirement: {}
  EnvVarRequirement:
    envDef:
      PYTHONPATH: /app
  ResourceRequirement:
    coresMax: 1
    ramMax: 512
hints:
  DockerRequirement:
    dockerPull: localhost/crop:latest
baseCommand: ["python", "-m", "app"]
arguments: []
inputs:
  item:
    type: string
    inputBinding:
      prefix: --input-item
  aoi:
    type: string
    inputBinding:
      prefix: --aoi
  epsg:
    type: string
    inputBinding:
      prefix: --epsg
  band:
    type: string
    inputBinding:
      prefix: --band
outputs:
  cropped:
    outputBinding:
      glob: '*.tif'
    type: File


Run the CWL description for the `crop` command line tool:

In [7]:


cwltool \
    --podman \
    --outdir ${WORKSPACE}/runs \
    ${WORKSPACE}/cwl-cli/crop.cwl \
    --item "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A" \
    --aoi="-121.399,39.834,-120.74,40.472" \
    --epsg "EPSG:4326" \
    --band "green" 2> crop.log

{
    "cropped": {
        "location": "file:///workspace/mastering-app-package/runs/crop_green.tif",
        "basename": "crop_green.tif",
        "class": "File",
        "checksum": "sha1$69255dfb77442b710fb7caf4fe2c555a8a8ca404",
        "size": 87481912,
        "path": "/workspace/mastering-app-package/runs/crop_green.tif"
    }
}


Let's look at the content of the stderr:

In [8]:
cat crop.log

INFO /opt/conda/bin/cwltool 3.1.20231114134824
INFO Resolved '/workspace/mastering-app-package/cwl-cli/crop.cwl' to 'file:///workspace/mastering-app-package/cwl-cli/crop.cwl'
WARNING [job crop] Skipping Docker software container '--memory' limit despite presence of ResourceRequirement with ramMin and/or ramMax setting. Consider running with --strict-memory-limit for increased portability assurance.
WARNING [job crop] Skipping Docker software container '--cpus' limit despite presence of ResourceRequirement with coresMin and/or coresMax setting. Consider running with --strict-cpu-limit for increased portability assurance.
INFO [job crop] /tmp/oib4vuzt$ podman \
    run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=/tmp/oib4vuzt,target=/xNTPND \
    --mount=type=bind,source=/tmp/z7ja0avk,target=/tmp \
    --workdir=/xNTPND \
    --read-only=true \
    --user=1001:100 \
    --rm \
    --cidfile=/tmp/nsa2zcr4/20241210164614-518471.cid \
    --env=TMPDIR=/tmp \
    --env=HOM

Remember the `cwltool` CWL runner can also take the parameters as a YAML file:


In [9]:
cat << EOF > params.yaml
item: https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A
aoi: "-121.399,39.834,-120.74,40.472"
epsg: "EPSG:4326"
band: nir
EOF

Invoke `crop` against the NIR band redirecting the stdout to a file:

In [ ]:
cwltool \
    --podman \
    --outdir ${WORKSPACE}/runs \
    ${WORKSPACE}/cwl-cli/crop.cwl \
    params.yaml > results.json 

Let's inspect the stdout produced. The `output` block with the id `cropped` of the CWL description is a file:

In [ ]:
cat results.json

In [ ]:
cat ${WORKSPACE}/cwl-cli/crop.cwl | yq e .outputs -

Let's list the results produced:

In [ ]:
ls -l ${RUNTIME}